In [ ]:
## define
bastion_ip='165.124.3.19'
bastion_user='admin'
bastion_pass='admin'
session="1"
dst_port="1"
src_port="2"

In [ ]:


import paramiko
import time
import re

def endwith_input(prompt_str,input_str,sleep=0,verbose=False):
    buff=''
    while not buff.endswith(prompt_str):
        resp = chan.recv(9999).decode("utf8")
        buff += str(resp)
    if verbose : print(buff)  
    time.sleep(sleep)
    chan.send(input_str+"\n")
    return buff


def endwith_exec_search(prompt_str, cmd, search_str, sleep=0, verbose=False):
    buff = ''
    while not buff.endswith(prompt_str):
        resp = chan.recv(9999).decode("utf8")
        buff += str(resp)
    if verbose : print("initb="+buff)  
    time.sleep(sleep)
    chan.send(cmd+ ";sleep 1; echo @@end@@"+"\n")
    while buff.find('@@end@@') < 0:
        resp = chan.recv(9999).decode("utf8")
        buff += str(resp)
    time.sleep(sleep)
    if verbose : print("buff="+buff)  
    if verbose : print("seach_str="+search_str)
    ret=re.search(search_str, buff)    
    if verbose : print("ret="+str(ret))
    if ret == None:
        return False
    else:
        return True
    #ret = True if ret != None else False


def exec_once(cmd):
    stdin, stdout, stderr = ssh.exec_command(cmd)
    for line in stdout:
        print('... ' + line.strip('\n'))

## initial 
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy( paramiko.AutoAddPolicy() )
ssh.connect(bastion_ip, username=bastion_user, password=bastion_pass)
chan = ssh.invoke_shell()

In [ ]:
## enable
endwith_input('# ','configure terminal', verbose=True)
endwith_input('# ','interface ethernet 1/1/'+dst_port, verbose=True)
endwith_input('# ','no switchport', verbose=True)
endwith_input('# ','exit', verbose=True)

endwith_input(')# ','monitor session '+session,verbose=True)

endwith_input(')# ','source interface ethernet 1/1/'+ src_port +' both',verbose=True)
endwith_input(')# ','destination interface ethernet 1/1/'+dst_port,verbose=True)
endwith_input(')# ','commit',verbose=True)
endwith_input(')# ','exit',verbose=True)
endwith_input('# ','show monitor session '+session,verbose=True)


In [ ]:
## check result
#ret=endwith_exec_search('# ','show monitor session all','ethernet',sleep=3,verbose=False)
#print("yes") if ret is True else print("not")

## show check command 
exec_once("show monitor session "+session)

In [ ]:
## tear down
ssh.connect(bastion_ip, username=bastion_user, password=bastion_pass)
chan = ssh.invoke_shell()
endwith_input('# ','configure terminal', verbose=True)
endwith_input(')# ','no monitor session '+session,verbose=True)
endwith_input(')# ','commit',verbose=True)
endwith_input(')# ','end',verbose=True)



In [ ]:
## check result
#ret=endwith_exec_search('# ','show monitor session all','ethernet',sleep=3,verbose=False)
#print("yes") if ret is True else print("not")

## show check command 
exec_once("show monitor session "+session)

In [ ]:
# close
ssh.close()